In [6]:
import time
import numpy as np
import json
from selenium import webdriver
import smtplib, ssl
import pickle
import datetime
import pyautogui

In [2]:
driver = webdriver.Firefox("/home/matt/Documents/Tools/Scraping/")

In [3]:
driver.get('https://www.freshdirect.com')

## Login To Fresh Direct, Navigate to Delivery Availability

This could further be automated but for simplicity, I am simply logging in once and navigating to the Delivery Availability Page

https://www.freshdirect.com/your_account/delivery_info_avail_slots.jsp

In [28]:
def move_to_element(driver, element):
    """This function attempts to move to the element on screen using Pyautogui.
    The scroll function above is similar but directly uses the selenium driver."""
    position = driver.get_window_rect()
    element_loc = element.location
    pyautogui.moveTo(position['x']+element_loc['x']+55, position['y']+115+element_loc['y'])

def click_element(driver, element):
    move_to_element(driver,element)
    time.sleep(0.1)
    pyautogui.click()

In [30]:
locbar = driver.find_element_by_id("locationbar")
locbar_right = locbar.find_element_by_xpath('//div[@class="locabar-right-sections"]')
class_ = "locabar-section locabar-user-section mouse"
xpath = '//div[@class="{}"]'.format(class_)
signOn = locbar_right.find_element_by_xpath(xpath)
# div = driver.find_element_by_id('locabar_user_login_link')
# signOn = div.find_element_by_xpath("//div[@class='locabar-arrow']")
click_element(driver, signOn)

In [31]:
email_field = driver.find_element_by_xpath('//input[@id="email"]')
for k in "mmitchell17@protonmail.com":
    email_field.send_keys(k)
    time.sleep(np.random.uniform(low=0.005, high=0.08))

In [33]:
password_field = driver.find_element_by_xpath('//input[@id="password"]')
for k in fresh_pass:
    password_field.send_keys(k)
    time.sleep(np.random.uniform(low=0.005, high=0.08))

In [40]:
signInBtn = driver.find_element_by_id("signinbtn")
# move_to_element(signInBtn)
signInBtn.click()

In [41]:
driver.get("https://www.freshdirect.com/your_account/delivery_info_avail_slots.jsp")

## Final Script

In [32]:
#Password Loading From Hidden File
def load_json(path):
    with open(path) as f:
        return json.load(f)
#Load Passwords
keys = load_json(".secret/mattbotkeys.json")
gmail_pass = keys['gmail']
fresh_pass = keys['fresh']
gEmail = keys['pEmail']
pEmail = keys['gEmail']

#Load Recipients
recipients = load_json(".secret/recipients.json")

def check_availability():
    """Checks Availability for Future Days (Does not check Today)
    Returns day, timeslot if availability found. Otherwise, returns None.
    """
    days = list(range(1,7))
    timeslots = list(range(6))
    for d in days:
        query = 'ts_d{}_hC'.format(d)
        header = driver.find_element_by_id(query)
        for t in timeslots:
            query = 'ts_d{}_ts{}_time'.format(d, t)
            time_slot = driver.find_element_by_id(query)
            attr = time_slot.get_attribute('class')
            ts = time_slot.text
            if 'Soldout' not in attr:
                return (header.text, ts)
    else:
        return None

def sendMessage(receiver_email, subject, message):
    message = 'Subject: {}\n\n{}'.format(subject, message)
    port = 465  # For SSL
    smtp_server = "smtp.gmail.com"
    sender_email = gEmail  # Enter your address
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, gmail_pass)
        server.sendmail(sender_email, receiver_email, message)



## Run Script

In [ ]:
try:
    while True:
        driver.refresh()
        try:
            availability = check_availability()
            if availability:
                day = availability[0]
                time_slot = availability[1]
                message = 'Delivery Availability on {} at {}!'.format(day, time_slot)
                message += """\n\n\n\nEmail sent from MattBot!"""
                for email in recipients:
                    sendMessage(email, "FreshDirect Delivery Available!", message)
                    time.sleep(5*60)
        except:
            sendMessage(pEmail, "Warning!","Hit Error.")
            now = datetime.datetime.now().strftime('%m-%d_%I-%M%p')
            driver.get_screenshot_as_file('ErrorPages/PageScreenshot_{}.png'.format(now))
            page_source = driver.page_source
            with open('ErrorPages/PageSourceCode_{}.pickle'.format(now), 'wb') as f:
                # Pickle the 'data' dictionary using the highest protocol available.
                pickle.dump(page_source, f, pickle.HIGHEST_PROTOCOL)
            time.sleep(60*5)

        time.sleep(np.random.uniform(low=30, high=100))
except:
    sendMessage(pEmail, "Warning!","Script crashed.")

## Full Script Beta

In [ ]:
while True:
    driver.refresh()
    days = list(range(1,7))
    timeslots = list(range(6))
    for d in days:
        query = 'ts_d{}_hC'.format(d)
        header = driver.find_element_by_id(query)
    #     print(header.text)
        for t in timeslots:
            query = 'ts_d{}_ts{}_time'.format(d, t)
            time_slot = driver.find_element_by_id(query)
            attr = time_slot.get_attribute('class')
            ts = time_slot.text
            if 'Soldout' not in attr:
                message = 'Delivery Availability on {} at {}!'.format(header.text, ts)
                port = 465  # For SSL
                smtp_server = "smtp.gmail.com"
                sender_email = gEmail  # Enter your address
                receiver_email = pEmail  # Enter receiver address
                message += """\n\nEmail sent from MattBot!"""

                context = ssl.create_default_context()
                with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
                    server.login(sender_email, gmail_pass)
                    server.sendmail(sender_email, receiver_email, message)
                break
    time.sleep(30)

## Twilio Connection

https://www.twilio.com/blog/2016/04/how-to-send-a-text-message-with-python.html

In [19]:
from twilio.rest import TwilioRestClient


In [ ]:
client = TwilioRestClient()

client.messages.create(from_='YOUR_NUMBER',
                       to='',
                       body='Ahoy from Twilio!')

## Gmail Connection

In [31]:
import smtplib, ssl

port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = gEmail  # Enter your address
receiver_email = pEmail # Enter receiver address
message = """Test email from MattBot!"""

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, gmail_pass)
    server.sendmail(sender_email, receiver_email, message)


## Initial Development Pieces

In [4]:
table = driver.find_element_by_tag_name('table')
table.text

''

In [42]:
days = list(range(1,7))
timeslots = list(range(6))
for d in days:
    query = 'ts_d{}_hC'.format(d)
    header = driver.find_element_by_id(query)
    print(header.text)
    for t in timeslots:
        query = 'ts_d{}_ts{}_time'.format(d, t)
        time_slot = driver.find_element_by_id(query)
        attr = time_slot.get_attribute('class')
        ts = time_slot.text
        print(ts, attr)
    print('\n\n')

TUE
Apr 21
6 am - 9 am fleft tsCont tsSoldoutC
9 am - 12 pm fleft tsCont tsSoldoutC
12 pm - 2 pm fleft tsCont tsSoldoutC
2 pm - 5 pm fleft tsCont tsSoldoutC
5 pm - 8 pm fleft tsCont tsSoldoutC
8 pm - 10 pm fleft tsCont tsSoldoutC



WED
Apr 22
6 am - 9 am fleft tsCont tsSoldoutC
9 am - 12 pm fleft tsCont tsSoldoutC
12 pm - 2 pm fleft tsCont tsSoldoutC
2 pm - 5 pm fleft tsCont tsSoldoutC
5 pm - 8 pm fleft tsCont tsSoldoutC
8 pm - 10 pm fleft tsCont tsSoldoutC



THU
Apr 23
6 am - 9 am fleft tsCont tsSoldoutC
9 am - 12 pm fleft tsCont tsSoldoutC
12 pm - 2 pm fleft tsCont tsSoldoutC
2 pm - 5 pm fleft tsCont tsSoldoutC
5 pm - 8 pm fleft tsCont tsSoldoutC
8 pm - 10 pm fleft tsCont tsSoldoutC



FRI
Apr 24
6 am - 9 am fleft tsCont tsSoldoutC
9 am - 12 pm fleft tsCont tsSoldoutC
12 pm - 2 pm fleft tsCont tsSoldoutC
2 pm - 5 pm fleft tsCont tsSoldoutC
5 pm - 8 pm fleft tsCont tsSoldoutC
8 pm - 10 pm fleft tsCont tsSoldoutC



SAT
Apr 25
6 am - 9 am fleft tsCont tsSoldoutC
9 am - 12 pm fleft ts